# stub for heatmap/pcolormesh tutorial
* surface dataset
+ heatmap
+ pcolormesh

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
im = np.load('data/dem.npy')

In [ ]:
type(im)

In [ ]:
plt.imshow(im)

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(im)
cbar = fig.colorbar(img)
cbar.set_label('elevation')

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(im, vmin=700, vmax=1000)
cbar = fig.colorbar(img)
cbar.set_label('elevation')

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(im, cmap='magma')
cbar = fig.colorbar(img)
cbar.set_label('elevation')

In [ ]:
fig, ax_arr = plt.subplots(2, 2)

for (ax, cm) in zip(ax_arr.flat, ['gray', 'magma', 'inferno', 'plasma']):
    img = ax.imshow(im, cmap=cm, origin='lower')
    fig

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True)
ax1.contour(im, cmap='magma')
ax1.set_aspect('equal')
ax2.imshow(im, cmap='magma', origin='lower')      

In [ ]:
import matplotlib.cm
list(matplotlib.cm.cmap_d)

In [ ]:
from matplotlib.colors import LogNorm

spike = np.abs(np.fft.fftshift(np.fft.fft2(im)))

fig, (ax1, ax2) = plt.subplots(1, 2, tight_layout=True, figsize=(10, 5))

lin_im = ax1.imshow(spike)
_ = fig.colorbar(lin_im, ax=ax1)
_ = ax1.set_title('linear color scale')

log_im = ax2.imshow(spike, norm=LogNorm())
_ = fig.colorbar(log_im, ax=ax2)
_ = ax2.set_title('log color scale')

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as mcm

cmap = mcm.tab20b
norm = mcolors.BoundaryNorm([1,3,5,7,9], cmap.N)

In [ ]:
data = np.arange(10).reshape((1,10))

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as mcm

cmap = mcm.tab20b
norm = mcolors.BoundaryNorm([1,3,5,7,9], cmap.N)

data = np.arange(10).reshape((1,10))
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.imshow(data, cmap=cmap)
ax2.imshow(data, norm=norm, cmap=cmap)